In [504]:
from datetime import datetime
from datetime import date
import pandas as pd

In [505]:
#define class for Period and Tenant
class Period:
    def __init__(self, name, start, end, fee):
        self.name = name
        self.start = start
        self.end = end
        self.start_r = self.start.strftime("%d %b %Y")
        self.end_r = self.end.strftime("%d %b %Y")
        self.len_days = (self.end - self.start).days

        self.fee = fee

        self.fully = []
        self.partially = []
        self.no_change = True

    def fully_present(self, t):
        if t.start <= self.start and t.end >= self.end:
            return True
        return False
    
    def partially_present(self, t):
        if self.fully_present(t) == False:
            if (t.start < self.end and t.start > self.start) or (t.end > self.start and t.end < self.end):
                return True
        return False
        
    
    #calculate fee per person for a period with no_change
    def calculate_per_person(self):
        return self.fee / len(self.fully)

class Tenant:
    def __init__(self, name, start, end=date.today()):
        self.name = name
        self.start = start
        self.end = end
        self.start_r = self.start.strftime("%d %b %Y")
        self.end_r = self.end.strftime("%d %b %Y")
        self.owes = 0

In [506]:
#define function to deal with overlap periods
def calculate_overlap_period(p):
    #set boundaries for sub_periods
    boundaries = []
    for t in p.partially:
        for date in (t.start, t.end):
            if date >= p.start and date <= p.end and date not in boundaries:
                boundaries.append(date)
    boundaries.append(p.start)
    boundaries.append(p.end)
    boundaries.sort()

    #split into smaller periods - add new_fee to relevant tenants
    for i in range(0, len(boundaries) - 1):
        new_start = boundaries[i]
        new_end = boundaries[i + 1]
        new_len = (new_end - new_start).days
        new_fee = (new_len / p.len_days) * p.fee
        sub_p = Period(f"{p.name}.{i}", new_start, new_end, new_fee)

        for t in all_tenants:
            if sub_p.fully_present(t) == True:
                sub_p.fully.append(t)
            if sub_p.partially_present(t) == True:
                sub_p.partially.append(t)
                sub_p.no_change = False
        if sub_p.no_change:
            for t in sub_p.fully:
                t.owes += sub_p.calculate_per_person()

In [507]:
#define periods
p1 = Period(1, date(2021, 5, 24), date(2021, 10, 27), 146.63)
p2 = Period(2, date(2021, 10, 28), date(2022, 3, 31), 136.52)
p3 = Period(3, date(2022, 4, 1), date(2022, 4, 26), 23.56)
p4 = Period(4, date(2022, 4, 27), date(2022, 10, 27), 202.04)
p5 = Period(5, date(2022, 10, 28), date(2023, 3, 31), 202.35)
p6 = Period(6, date(2023, 4, 1), date(2023, 4, 12), 17.46)

all_periods = [p1, p2, p3, p4, p5, p6]

for p in all_periods:
    print(f"Fee for period {p.name} -> {p.start_r} to {p.end_r} ({p.len_days} days) \t= £{p.fee}")

Fee for period 1 -> 24 May 2021 to 27 Oct 2021 (156 days) 	= £146.63
Fee for period 2 -> 28 Oct 2021 to 31 Mar 2022 (154 days) 	= £136.52
Fee for period 3 -> 01 Apr 2022 to 26 Apr 2022 (25 days) 	= £23.56
Fee for period 4 -> 27 Apr 2022 to 27 Oct 2022 (183 days) 	= £202.04
Fee for period 5 -> 28 Oct 2022 to 31 Mar 2023 (154 days) 	= £202.35
Fee for period 6 -> 01 Apr 2023 to 12 Apr 2023 (11 days) 	= £17.46


In [508]:
#define tenants
chris = Tenant("Christian", date(2021, 5, 24))
cam = Tenant("Cameron", date(2021, 5, 24))
mikey = Tenant("Michael", date(2021, 5, 24), date(2022, 9, 24))
ren = Tenant("Renata", date(2022, 5, 24))
jack = Tenant("Jack", date(2022, 9, 24))

all_tenants = [chris, cam, mikey, ren, jack]

for t in all_tenants:
    end = "present" if t.end == date.today() else t.end_r
    print(f"{t.name} lived here from {t.start_r} until {end}")


Christian lived here from 24 May 2021 until present
Cameron lived here from 24 May 2021 until present
Michael lived here from 24 May 2021 until 24 Sep 2022
Renata lived here from 24 May 2022 until present
Jack lived here from 24 Sep 2022 until present


In [509]:
#for each period, populate fully[] and partially[]
def populate_fully_partially(periods, tenants):
    for p in periods:
        for t in tenants:
            if p.fully_present(t) == True:
                p.fully.append(t)
            if p.partially_present(t) == True:
                p.partially.append(t)
                p.no_change = False

populate_fully_partially(all_periods, all_tenants)
#UNCOMMENT FOR TESTING:
#for p in all_periods:
#    print(f"For period {p.name}: partially = {[x.name for x in p.partially]}; fully = {[x.name for x in p.fully]}")

In [510]:
#calculate owings and print totals
#calculate rent if no_change, else deal with overlap period
def calculate_owings():
    for p in all_periods:
        if p.no_change:
            for t in p.fully:
                t.owes += p.calculate_per_person()
        else:
            calculate_overlap_period(p)

def print_totals():
    total = 0
    for t in all_tenants:
        total += t.owes
        print(f"{t.name} owes {t.owes}")
    print(f"total = {total}")

calculate_owings()
print_totals()

Christian owes 210.18326502732242
Cameron owes 210.18326502732242
Michael owes 146.1224043715847
Renata owes 98.01020491803278
Jack owes 64.0608606557377
total = 728.56
